In [ ]:
import datetime as dt
from typing import NamedTuple

from micro_namedtuple_sqlite_persister.persister import Engine, register_serde


In [ ]:
class MyModel(NamedTuple):
    id: int | None
    name: str
    date: dt.datetime
    score: float

In [ ]:
# engine = Engine(":memory:")
engine = Engine("example.db")
engine.ensure_table_created(MyModel)

In [ ]:
row = MyModel(None, "foo", dt.datetime.now(), 6.5)
row = engine.insert(row)
row

In [ ]:
engine.get(MyModel, row.id)

In [ ]:
from random import random
randfloat = random()
engine.update(row._replace(score=row.score*randfloat))
engine.get(MyModel, row.id)

In [ ]:
row2 = engine.insert(MyModel(None, "foo", dt.datetime.now(), 6.5))
print(row2)

print(f"\nAbout to delete the recently insert row.id: {row2.id}")
engine.delete(MyModel, row2.id)

In [ ]:
# Insert many

for i in range(5000):
    engine.insert(MyModel(None, "foo", dt.datetime.now(), random()*100))

engine.connection.commit()

In [ ]:
# update many
for i in range(5000):
    engine.update(MyModel(i+1, "foo", dt.datetime.now(), random()*100))

engine.connection.commit()

# Error Cases

In [ ]:
# inserting a row with an id that already exists will raise an error
engine.insert(MyModel(row.id, "bar", dt.datetime.now(), 3.14))

In [ ]:
# Trying to update a row without specifying an id will raise an error
engine.update(MyModel(None, "bar", dt.datetime.now(), 3.14))

In [ ]:
# Raises an error if the id does not exist
engine.update(MyModel(878787879879, "bar", dt.datetime.now(), 3.14))

# Custom Serialization/Deserialization

In [ ]:
from io import BytesIO
import pandas as pd

from micro_namedtuple_sqlite_persister.persister import register_serde

def serialize_df(obj: pd.DataFrame) -> bytes:
    return obj.to_parquet()


def deserialize_df(data: bytes) -> pd.DataFrame:
    return pd.read_parquet(BytesIO(data))


register_serde(serialize_df, deserialize_df)

